In [1]:
import surprise
from surprise import KNNWithZScore
from surprise import Reader, Dataset
import pandas as pd
import numpy as np

In [5]:
from google.colab import files
ratings = files.upload()

Saving zomato_restaurents_reviews.xlsx to zomato_restaurents_reviews.xlsx


In [2]:
ratings = pd.read_csv('zomato_scrap.csv')
ratings.head()

,rating,user_id,user_name,rest_name
0,Rated 4.0,16214943.0,Khushbu Rithal,Avista Cafe Pizzeria Bangalore
1,Rated 5.0,7225951.0,Vishnu Das,Avista Cafe Pizzeria Bangalore
2,Rated 5.0,501410.0,Saunak Mukherjee,Avista Cafe Pizzeria Bangalore
3,Rated 4.0,27258850.0,Abhinaya,Avista Cafe Pizzeria Bangalore
4,Rated 4.0,83298879.0,ROSHNI DESHMUKH,Avista Cafe Pizzeria Bangalore


In [7]:
new = ratings["rating"].str.split(" ", n = 1, expand = True) 
  
# making seperate last name column from new data frame 
ratings["rating_new"]= new[1] 
  
# Dropping old Name columns 
ratings.drop(columns =["rating"], inplace = True) 

In [9]:
ratings["rating"]= ratings["rating_new"] 
  
# Dropping old Name columns 
ratings.drop(columns =["rating_new"], inplace = True)

In [10]:
ratings.head()

,user_id,user_name,rest_name,rating
0,16214943.0,Khushbu Rithal,Avista Cafe Pizzeria Bangalore,4.0
1,7225951.0,Vishnu Das,Avista Cafe Pizzeria Bangalore,5.0
2,501410.0,Saunak Mukherjee,Avista Cafe Pizzeria Bangalore,5.0
3,27258850.0,Abhinaya,Avista Cafe Pizzeria Bangalore,4.0
4,83298879.0,ROSHNI DESHMUKH,Avista Cafe Pizzeria Bangalore,4.0


In [4]:
#restaurents = ratings[['Restaurent_ID', 'Restaurent_Name']].drop_duplicates().set_index('Restaurent_ID')
#restaurents.head()

In [11]:
reader = Reader(rating_scale = (1,5))
data = Dataset.load_from_df(ratings[['user_id','rest_name', 'rating']], reader)
sim_options = {
    'name': 'cosine',
    'user_based': True,
    'k':10
}

knn = KNNWithZScore(sim_options = sim_options)
knn.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [12]:
knn.predict(uid = 37440297, iid = 18426381, verbose = True)

user: 37440297   item: 18426381   r_ui = None   est = 5.00   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}


Prediction(uid=37440297, iid=18426381, r_ui=None, est=5, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

In [15]:
def Prediction(userid):
    predictions = pd.DataFrame()

    for restaurentid in ratings['Restaurent_Name'].unique():
      restaurent_row = ratings[(ratings['User_ID'] == userid) & (ratings['Restaurent_Name'] == restaurentid)]
      if len(restaurent_row) == 0:
        restaurent_pred_rating = knn.predict(uid = userid, iid = restaurentid)[3]
        predictions = predictions.append({'Restaurent_Name':restaurentid, 'rating_prediction':restaurent_pred_rating}, ignore_index = True)
    ids = predictions.sort_values('rating_prediction', ascending = False).head(3)['Restaurent_Name'].astype('int')
    return restaurents.loc[ids]['Restaurent_Name']

In [16]:
Prediction(37440297)

KeyError: 'Restaurent_Name'